In [ ]:
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from kafka import KafkaProducer

In [ ]:
spark = SparkSession.builder.master("local[*]").appName('ex5_reviews_producer').getOrCreate()

In [ ]:
data_df = spark.read.parquet('hdfs://course-hdfs:8020/data/source/google_reviews')

In [ ]:
data = data_df.toJSON()

In [ ]:
producer = KafkaProducer(bootstrap_servers='course-kafka:9092', value_serializer=lambda v: v.encode('utf-8'))

In [ ]:
i = 0

for json_data in data.collect():
    i = i + 1
    producer.send(topic='gps-user-review-source', value=json_data)
    if i == 50:
        producer.flush()
        time.sleep(5)
        i = 0

In [ ]:
producer.close()

In [ ]:
spark.stop()